In [3]:
import os, glob, shlex, subprocess, textwrap

protein_pdb = "/home/aashish/cholestrol/pz2_cholesterol/protein.pdb"
folder1 = sorted(glob.glob("/home/aashish/cholestrol/pz2_cholesterol/6.sdf_2_pdb/*.pdb"))

out_png = "scene.png"
pml = ["reinitialize"]

# Performance: defer builds while bulk loading, silence chatter, avoid auto-zoom
pml += [
    "set defer_builds_mode, 3",
    "set suspend_updates, 1",
    "set auto_zoom, 0",
    "feedback disable, all, actions",
    "feedback disable, all, results",
    "set stick_quality, 8",
]

# Load protein & style
pml += [
    f'load "{protein_pdb}", protein',
    "hide everything, protein",
    "show cartoon, protein",
    "color slate, protein",
]

# Load each small molecule as its own object, style as sticks
for f in folder1:
    obj = "set1_" + os.path.splitext(os.path.basename(f))[0]
    pml += [
        f'load "{f}", {obj}',
        f"hide everything, {obj}",
        f"show sticks, {obj}",
        f"set stick_radius, 0.15, {obj}",
    ]

# Finish builds once
pml += [
    "set defer_builds_mode, 0",
    "set suspend_updates, 0",
    "rebuild",
    "group folder1_group, set1_*",
    "bg_color white",
    "orient protein",
]

# Render and quit
pml += [
    "ray 1600, 1200",
    f'png "{out_png}", dpi=300',
    "quit",
]

script_path = "bulk_load_render.pml"
with open(script_path, "w") as fh:
    fh.write("\n".join(pml) + "\n")

# Run PyMOL headless (-c) and quiet (-q)
# If 'pymol' isn't on PATH, replace 'pymol' with the full path to the binary.
cmd = f'pymol -cq {shlex.quote(script_path)}'
print("Running:", cmd)
res = subprocess.run(cmd, shell=True)
print("Return code:", res.returncode)

if os.path.exists(out_png):
    print("Rendered:", os.path.abspath(out_png))
else:
    print("No PNG produced; check terminal output for PyMOL errors.")


Running: pymol -cq bulk_load_render.pml
Return code: 127
No PNG produced; check terminal output for PyMOL errors.


/bin/sh: line 1: pymol: command not found
